When compiling by Rosa Martinez Corral, the original code produced steady-state results different than those obtained for python with the Ladder 3 model. It turns out that the problem was that the for loop where this was being executed read for (int i; i<n_states-1; i++). When instead this is fixed to for (int i=0; i<n_states-1; i++) the results coincide. This is illustrated here:

In [1]:
import numpy as np
import sys, os

In [2]:
#first of all, compile the code where the only change has been this (and the classes are also named differently to make sure there are no clashes when importing both of them together)
path_to_eigen="/Users/rosamartinezcorral/Documents/eigenlibrary/eigen-eigen-323c052e1731/" 
path_to_FPTrepo="\"../\""
file="Ladder_3_i0.cpp"
folder="../bin"
path_1=os.path.join("../", "src")
fname=os.path.join(path_1, file)
objectnamemac=os.path.join(folder,file.replace('.cpp','')) 

compilestringmac="c++ -O2 -DNDEBUG -Wall -shared -std=c++11  -fPIC -undefined dynamic_lookup -I %s -I %s -lmpfr -lmpc `python3 -m pybind11 --includes` %s -o %s`python3-config --extension-suffix`"%(path_to_FPTrepo,path_to_eigen, fname,objectnamemac)

compilestring=compilestringmac
print(compilestring)

#!$compilestring

c++ -O2 -DNDEBUG -Wall -shared -std=c++11  -fPIC -undefined dynamic_lookup -I "../" -I /Users/rosamartinezcorral/Documents/eigenlibrary/eigen-eigen-323c052e1731/ -lmpfr -lmpc `python3 -m pybind11 --includes` ../src/Ladder_3_i0.cpp -o ../bin/Ladder_3_i0`python3-config --extension-suffix`


In [3]:
from scipy.linalg import null_space
sys.path.insert(0, "../bin")
from Ladder_3_i0 import Ladder_3_i0 as L3i0
from Ladder_3 import Ladder_3 as L3original
from Ladder_3_v2 import Ladder_3_v2 as L3v2

In [4]:
#python calculation of the steady-state
def compute_ss_fromL(L):
    L_d=np.diag(np.sum(L,axis=0))
    L=L-L_d
    rhos=null_space(L) #column vector
    rhos=np.transpose(rhos)[0] #1 row
    ss=np.abs(rhos)/np.sum(np.abs(rhos)) #can be - when very close to 0
    #ss=rhos/np.sum(rhos)
    return ss
def ss_Ladder3_python(cpp_param, x):
    a12, a23, b12, b23, a12T, a23T, b12T, b23T, kon, koff, P, PT=cpp_param
    #1<->2<->3 : transitions without TF bound
    #4<->5<->6: transition with TF bound
    #(i,j) contains the label for the transition from j+1 to i+1 (considering nodes are indexed 1-6 not 0-5)
    L_Ladder3=np.array([[0, b12, 0, koff, 0, 0],
                       [a12, 0, b23, 0, koff, 0],
                       [0, a23, 0, 0, 0, koff],
                       [kon*x,0,0,0,b12T,0],
                        [0, kon*x, 0, a12T, 0, b23T],
                        [0,0,kon*x, 0, a23T, 0]])
    
    return compute_ss_fromL(L_Ladder3)


In [5]:
obji0 = L3i0()
objGR = L3original()
objv2 = L3v2()

In [8]:
#look at the ss of a given parameter set
a12=0.1
b12=0.01
a23=0.1
b23=0.01
a12T=a12*10
a23T=a23*10
b12T=b12
b23T=b23
kon=1
koff=1
print(koff+b23T)
for tf_conc in [3, 1000, 10**15]:
    for P in [0, 1, 1000]:
        print("tf_conc=", tf_conc, "P=", P)
        pars=np.array([a12,a23,b12,b23,a12T,a23T,b12T,b23T,kon,koff,P, P])
        obji0.setLaplacian_ladder_3(pars) # super().setLaplacian_ladder_3(converted_pset)-> set_ladder_3_laplacian-> fills laplacian attribute
        objGR.setLaplacian_ladder_3(pars)
        objv2.setLaplacian_ladder_3(pars)
        #print(obj.getLaplacian())

        ss_python=ss_Ladder3_python(pars,tf_conc)
        r_python=ss_python[2]+ss_python[5]
        print("python:", r_python)
        for o,obj in enumerate([obji0, objGR, objv2]):
            if o==0:
                print("int i (original)", end=":")
            elif o==1:
                print("int i=0 (fixed)", end=":")
            elif o==2:
                print("abs", end=":")
            
            if True:
                
                r_c=obj.compute_response(tf_conc) # compute ss response 

                if np.abs(r_python-r_c)>0.01:
                    print("**different response", o, r_c, end=" |")

                else:
                    print("same response", end=" |")

        print("\n ============")

1.01
tf_conc= 3 P= 0
python: 0.9864961470966014
int i (original):same response |int i=0 (fixed):same response |abs:same response |
tf_conc= 3 P= 1
python: 0.9864961470966014
int i (original):same response |int i=0 (fixed):**different response 1 0.3464330610518742 |abs:same response |
tf_conc= 3 P= 1000
python: 0.9864961470966014
int i (original):same response |int i=0 (fixed):**different response 1 0.00044602832239432576 |abs:same response |
tf_conc= 1000 P= 0
python: 0.9899919855601608
int i (original):same response |int i=0 (fixed):same response |abs:same response |
tf_conc= 1000 P= 1
python: 0.9899919855601608
int i (original):same response |int i=0 (fixed):**different response 1 0.44077717950657946 |abs:same response |
tf_conc= 1000 P= 1000
python: 0.9899919855601608
int i (original):same response |int i=0 (fixed):**different response 1 0.000612744894123417 |abs:same response |
tf_conc= 1000000000000000 P= 0
python: 0.004889061685256302
int i (original):**different response 0 0.990

Note that python fails for large tf_conc.